In [1]:
import find_eoms
import find_A_theta

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, Dataset, DataLoader

from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [2]:
import numpy as np

# Number of samples
q1_low  = -torch.pi
q1_high =  torch.pi
q2_low  = -torch.pi
q2_high =  torch.pi

n_samples = 100000

# Generate uniformly distributed points for q1 and q2
#q1 = torch.random.uniform(low=-10, high=10, size=n_samples)
#q2 = torch.random.uniform(low=-10, high=10, size=n_samples)
q1 = torch.linspace(q1_low, q1_high, n_samples)
q2 = torch.linspace(q2_low, q2_high, n_samples)

idx = torch.randperm(q2.shape[0])

q2 = q2[idx]

# Stack q1 and q2 to get the 2D coordinates
points = torch.stack([q1, q2], axis=1)  # Shape will be (1000, 2)

In [3]:
# Use TensorDataset to create the dataset
dataset = TensorDataset(points)

# Create the DataLoader with batch size and shuffling
batch_size = 128
dataloader = DataLoader(dataset, 
                        batch_size=batch_size, 
                        shuffle=True,
                        num_workers=0,
                        pin_memory=True)

print(dataloader)

# Example usage: iterate through the DataLoader
for batch in dataloader:
    #print(batch)
    break  # Just to show one batch

In [4]:
dataiter = iter(dataloader)
coordinates = next(dataiter)
print(coordinates)


[tensor([[-2.0480, -2.1489],
        [ 2.6768,  1.0079],
        [-1.4018, -2.1268],
        [-2.2396, -1.4525],
        [-1.7045,  1.8024],
        [ 1.9638, -1.9231],
        [ 0.3447, -2.2822],
        [ 2.4770,  2.4592],
        [-1.4361, -0.3036],
        [-0.1190,  0.5697],
        [-2.1402, -0.6967],
        [ 2.2182,  2.1447],
        [ 2.6517,  1.7793],
        [ 1.5443,  2.3397],
        [ 3.1204,  2.2164],
        [-2.2303,  0.2414],
        [-0.0124, -2.1783],
        [ 0.7703, -0.5350],
        [ 1.7048, -1.3444],
        [-1.8324,  2.1563],
        [ 1.4141, -1.2910],
        [ 2.3783,  1.2583],
        [-1.6587, -2.5151],
        [-0.2429, -2.0076],
        [ 1.8967, -1.2446],
        [ 1.2187,  2.7638],
        [ 2.7267, -1.3686],
        [-2.9397,  2.6969],
        [-0.6252, -1.7470],
        [-0.7166,  1.0588],
        [-3.0771,  1.0473],
        [ 2.3167, -0.9687],
        [-2.4570, -2.7471],
        [-0.1275, -3.0625],
        [-0.9470,  0.3104],
        [ 2.2949,  

In [5]:
class SinCosLayer(nn.Module):
    def __init__(self):
        super(SinCosLayer, self).__init__()

    def forward(self, x):
        # Apply sin() and cos() to both coordinates
        x_sin = torch.sin(x)
        x_cos = torch.cos(x)
        x_sin_cos_shape = (x.shape[0], x.shape[1]*2)
        x_sin_cos = torch.empty(x_sin_cos_shape, dtype=x_sin.dtype, device=x.device)
        x_sin_cos[:,0::2] = x_sin
        x_sin_cos[:,1::2] = x_cos
        return x_sin_cos
    
# Custom layer to reverse the interleaved sin() and cos() back to original coordinates
class InverseSinCosLayer(nn.Module):
    def __init__(self):
        super(InverseSinCosLayer, self).__init__()

    def forward(self, x):
        # x contains interleaved sin() and cos() values
        # Assuming input is of shape (batch_size, 4) for 2D coordinates
        sin_vals = x[:, 0::2]  # Extract sin values
        cos_vals = x[:, 1::2]  # Extract cos values

        # Use atan2 to recover the original angles from sin and cos
        original_coords = torch.atan2(sin_vals, cos_vals)
        return original_coords
    



class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            #SinCosLayer(),
            nn.Linear(2, 16),
            nn.ReLU(),
            nn.Linear(16, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 2)#,
            #InverseSinCosLayer()
        )
        
        self.decoder = nn.Sequential(
            #SinCosLayer(),
            nn.Linear(2, 16),
            nn.ReLU(),
            nn.Linear(16, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 2)#,
            #InverseSinCosLayer()
        )
    
    def forward(self, q):
        theta = self.encoder(q)
        q_hat = self.decoder(theta)
        return(theta, q_hat)

In [6]:
%%time


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Autoencoder().to(device)  # Move model to GPU

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
#scheduler.get_last_lr()


num_epochs = 50
outputs = []
for epoch in range(num_epochs):
    for (q) in dataloader:
        q = q[0].to(device)
        Jh_learned, q_hat = model(q)
        Jh = 
        loss_reconstruction = criterion(q_hat, q)
        loss_theta = criterion(theta, q)
        loss = loss_reconstruction + loss_theta
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    scheduler.step()
    #print(scheduler.get_last_lr())
        
    print(f'Epoch:{epoch+1}, Loss:{loss.item():.7f}')
    outputs.append((epoch, q, q_hat, theta))

Epoch:1, Loss:0.0007325
Epoch:2, Loss:0.0002668
Epoch:3, Loss:0.0002301
Epoch:4, Loss:0.0001047
Epoch:5, Loss:0.0000737
Epoch:6, Loss:0.0000662
Epoch:7, Loss:0.0000571
Epoch:8, Loss:0.0000419
Epoch:9, Loss:0.0000205
Epoch:10, Loss:0.0000355
Epoch:11, Loss:0.0000178
Epoch:12, Loss:0.0000227
Epoch:13, Loss:0.0000643
Epoch:14, Loss:0.0000185
Epoch:15, Loss:0.0000111
Epoch:16, Loss:0.0000419
Epoch:17, Loss:0.0000251
Epoch:18, Loss:0.0000179
Epoch:19, Loss:0.0000143
Epoch:20, Loss:0.0000074
Epoch:21, Loss:0.0000184
Epoch:22, Loss:0.0000135
Epoch:23, Loss:0.0000136
Epoch:24, Loss:0.0000088
Epoch:25, Loss:0.0000050
Epoch:26, Loss:0.0000136
Epoch:27, Loss:0.0000121
Epoch:28, Loss:0.0000087
Epoch:29, Loss:0.0000066
Epoch:30, Loss:0.0000146
Epoch:31, Loss:0.0000084
Epoch:32, Loss:0.0000107
Epoch:33, Loss:0.0000120
Epoch:34, Loss:0.0000063
Epoch:35, Loss:0.0000058
Epoch:36, Loss:0.0000057
Epoch:37, Loss:0.0000044
Epoch:38, Loss:0.0000050
Epoch:39, Loss:0.0000048
Epoch:40, Loss:0.0000095
Epoch:41,

In [7]:
print("q\n", outputs[-1][1][0:4])
print("q_hat\n", outputs[-1][2][0:4])
print("theta\n", outputs[-1][3][0:4])
print("\n\n")
print("q_hat - q\n", outputs[-1][2][0:4] - outputs[-1][1][0:4])
print("theta - q\n", outputs[-1][3][0:4] - outputs[-1][1][0:4])

q
 tensor([[-2.4975,  0.9781],
        [-2.9574,  0.0200],
        [ 1.6092, -2.1312],
        [-2.5148,  0.2547]], device='cuda:0')
q_hat
 tensor([[-2.4984,  0.9780],
        [-2.9562,  0.0199],
        [ 1.6086, -2.1333],
        [-2.5155,  0.2568]], device='cuda:0', grad_fn=<SliceBackward0>)
theta
 tensor([[-2.4980,  0.9786],
        [-2.9562,  0.0183],
        [ 1.6110, -2.1310],
        [-2.5144,  0.2544]], device='cuda:0', grad_fn=<SliceBackward0>)



q_hat - q
 tensor([[-8.6737e-04, -4.5002e-05],
        [ 1.1873e-03, -1.6190e-04],
        [-5.2762e-04, -2.1198e-03],
        [-6.9189e-04,  2.1041e-03]], device='cuda:0', grad_fn=<SubBackward0>)
theta - q
 tensor([[-0.0005,  0.0005],
        [ 0.0012, -0.0017],
        [ 0.0019,  0.0002],
        [ 0.0004, -0.0003]], device='cuda:0', grad_fn=<SubBackward0>)


In [8]:
for param in model.parameters():
  print(param.data)

tensor([[ 0.7252,  0.3208],
        [-0.6015, -0.2684],
        [ 0.4316, -0.5483],
        [ 0.1033,  0.4212],
        [ 0.3648, -0.0747],
        [-0.3016, -0.6840],
        [ 0.2600,  0.5876],
        [ 0.0598,  0.7455],
        [ 0.2194,  0.4443],
        [-0.7427, -0.1985],
        [-0.6463,  0.6342],
        [-0.2876, -0.1439],
        [-0.1870, -0.7647],
        [ 0.5872,  0.3832],
        [-0.7214,  0.1464],
        [-0.5636,  0.7176]], device='cuda:0')
tensor([ 0.3259, -0.2729, -0.5178, -0.2353, -0.2777, -0.1664,  0.1394,  0.6873,
         0.3512,  0.0120,  0.1424, -0.0488,  0.4247, -0.0366,  0.5441,  0.6804],
       device='cuda:0')
tensor([[ 0.3607, -0.1063,  0.2640,  ..., -0.1416,  0.0387, -0.1803],
        [ 0.0175,  0.1469,  0.3106,  ..., -0.0418, -0.0346, -0.2877],
        [ 0.0477,  0.0590,  0.1373,  ..., -0.0773,  0.2205,  0.2077],
        ...,
        [ 0.3066,  0.2616,  0.2686,  ...,  0.0932, -0.0760, -0.1024],
        [-0.0808,  0.2074, -0.0358,  ...,  0.1773,  0.05

In [9]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[ 0.7252,  0.3208],
        [-0.6015, -0.2684],
        [ 0.4316, -0.5483],
        [ 0.1033,  0.4212],
        [ 0.3648, -0.0747],
        [-0.3016, -0.6840],
        [ 0.2600,  0.5876],
        [ 0.0598,  0.7455],
        [ 0.2194,  0.4443],
        [-0.7427, -0.1985],
        [-0.6463,  0.6342],
        [-0.2876, -0.1439],
        [-0.1870, -0.7647],
        [ 0.5872,  0.3832],
        [-0.7214,  0.1464],
        [-0.5636,  0.7176]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.3259, -0.2729, -0.5178, -0.2353, -0.2777, -0.1664,  0.1394,  0.6873,
         0.3512,  0.0120,  0.1424, -0.0488,  0.4247, -0.0366,  0.5441,  0.6804],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 0.3607, -0.1063,  0.2640,  ..., -0.1416,  0.0387, -0.1803],
        [ 0.0175,  0.1469,  0.3106,  ..., -0.0418, -0.0346, -0.2877],
        [ 0.0477,  0.0590,  0.1373,  ..., -0.0773,  0.2205,  0.2077],
        ...,
        [ 0.3066,  0